In [8]:
import xgboost
import pandas as pd
from xgboost import XGBRegressor
import sklearn
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
import random
from sklearn.multioutput import MultiOutputRegressor
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, BayesianRidge
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import KFold
import pyDOE2
import optuna
from tqdm import tqdm
import datetime
import time

import tensorflow as tf


import sys
sys.path.append('../structural')

import evaluation
import getInvalid
import DataAugment

import importlib
importlib.reload(DataAugment)
importlib.reload(evaluation)
importlib.reload(getInvalid)

from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.factory import get_problem
from pymoo.optimize import minimize
from pymoo.visualization.scatter import Scatter
from pymoo.core.problem import ElementwiseProblem

In [9]:
data=pd.read_csv("all_structural_data_oh.csv", index_col=0)
batch=data.iloc[:,-1]
x_orig = data.iloc[:,:-11]
y_orig = data.iloc[:,-11:-1]
# print(x_orig)
# print(y_orig)

xscaler = StandardScaler()
xscaler.fit(x_orig)
x_scaled = xscaler.transform(x_orig)
x_scaled=np.float32(x_scaled)

#Take Absolute value of columns that where we try to minimize magnitude and take 
y=y_orig.copy()
for col in ['Sim 1 Safety Factor', 'Sim 3 Safety Factor']:
    y[col]=1/y[col]
for col in ['Sim 1 Dropout X Disp.', 'Sim 1 Dropout Y Disp.', 'Sim 1 Bottom Bracket X Disp.', 'Sim 1 Bottom Bracket Y Disp.', 'Sim 2 Bottom Bracket Z Disp.', 'Sim 3 Bottom Bracket Y Disp.', 'Sim 3 Bottom Bracket X Rot.', 'Model Mass']:      
    y[col]=[np.abs(val) for val in y[col].values]
# y=np.float32(y.values)

y_scaler = StandardScaler()
y_scaler.fit(y.values)
y_scaled = y_scaler.transform(y.values)


In [44]:
df=pd.DataFrame(x_scaled)
lowers=df.quantile(0.02).values
uppers=df.quantile(0.98).values
print(lowers)
print(xscaler.inverse_transform([lowers]))
print(xscaler.inverse_transform([uppers]))

[-1.20232546 -0.47357535 -0.53991389 -0.82155234 -0.79936159 -1.39574283
 -2.48103356 -2.31542683 -0.92990398 -2.06096911 -1.35411406 -0.99316508
 -1.88219893 -1.11949182 -1.6191572  -1.18864572 -1.28995001 -2.83453524
 -1.76873953 -0.93762004 -1.5656116  -2.32862759 -1.43197174 -2.5569734
 -0.20951992 -1.49756002 -2.1704061  -3.60928679 -0.10460914 -1.45389056
 -1.52720535 -1.03802431 -1.03252304 -1.02995622 -1.02619441 -1.02331945
 -1.02977089 -1.03404555 -2.24665977]
[[-1.28013741e-08 -3.18982279e-09  4.91113722e-09 -2.91439262e-10
  -4.81304196e-09  3.33992799e-01 -2.69999997e-02  5.03600003e-01
   1.85500011e-02  6.70000001e+01  3.50000000e-02  2.53999999e-02
   3.17000000e-02  3.18263998e-02  1.71000001e-02  1.04999999e-02
   2.72000000e-02  1.09679999e-02  2.83760001e-02  1.25000012e-02
   1.68999999e-02  6.44999999e+01  8.00988004e-02  2.39960002e-01
   6.80000000e-02  1.20000000e-01  1.34000000e-02  1.50000000e-02
   3.30000000e-01  3.00000000e-01  7.00000004e-03  5.29880078e-

In [87]:
regressor=tf.keras.models.load_model("./Trained Models/BestNN")
regressor.trainable=False
classifier=tf.keras.models.load_model("./Trained Models/BestNN_Classifier_final")
classifier.trainable=False
class build_problem(ElementwiseProblem):
    def __init__(self, lowers, uppers):
        super().__init__(n_var=39,
                         n_obj=10,
                         n_constr=1,
                         xl=lowers,
                         xu=uppers)
    def _evaluate(self, x, out, *args, **kwargs):
        x=np.array([x])
        invalidprob=classifier.predict(x)
        out["F"] = regressor.predict(x)
        out["G"] = [invalidprob[0][0]-0.1]
        
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.factory import get_sampling, get_crossover, get_mutation
from pymoo.factory import get_termination

termination = get_termination("n_gen", 200)
algorithm = NSGA2(
    pop_size=250,
    n_offsprings=10,
    sampling=get_sampling("real_random"),
    crossover=get_crossover("real_sbx", prob=0.9, eta=15),
    mutation=get_mutation("real_pm", eta=20),
    eliminate_duplicates=True
)

In [88]:
from pymoo.optimize import minimize
problem = build_problem(lowers, uppers)

print(problem)
res = minimize(problem,
               algorithm,
               termination,
               seed=1,
               save_history=True,
               verbose=True)

# name: build_problem
# n_var: 39
# n_obj: 10
# n_constr: 1

n_gen |  n_eval |   cv (min)   |   cv (avg)   |  n_nds  |     eps      |  indicator  
    1 |     250 |  0.00000E+00 |  0.008530316 |      63 |            - |            -
    2 |     260 |  0.00000E+00 |  0.000423020 |      64 |  0.066791628 |        ideal
    3 |     270 |  0.00000E+00 |  0.000010798 |      66 |  0.003262425 |            f
    4 |     280 |  0.00000E+00 |  0.00000E+00 |      69 |  0.004873249 |            f
    5 |     290 |  0.00000E+00 |  0.00000E+00 |      73 |  0.048262401 |        ideal
    6 |     300 |  0.00000E+00 |  0.00000E+00 |      74 |  0.000451283 |            f
    7 |     310 |  0.00000E+00 |  0.00000E+00 |      76 |  0.004332058 |            f
    8 |     320 |  0.00000E+00 |  0.00000E+00 |      75 |  0.297235012 |        nadir
    9 |     330 |  0.00000E+00 |  0.00000E+00 |      78 |  0.005733457 |            f


C:\Users\Lyle\anaconda3\envs\tf27\lib\site-packages\pymoo\operators\mutation\pm.py:34: RuntimeWarning: invalid value encountered in true_divide
  delta1 = (X - xl) / (xu - xl)
C:\Users\Lyle\anaconda3\envs\tf27\lib\site-packages\pymoo\operators\mutation\pm.py:35: RuntimeWarning: invalid value encountered in true_divide
  delta2 = (xu - X) / (xu - xl)


   10 |     340 |  0.00000E+00 |  0.00000E+00 |      78 |  0.007166056 |            f
   11 |     350 |  0.00000E+00 |  0.00000E+00 |      81 |  0.742793075 |        nadir
   12 |     360 |  0.00000E+00 |  0.00000E+00 |      83 |  0.005167541 |            f
   13 |     370 |  0.00000E+00 |  0.00000E+00 |      87 |  0.075084135 |        ideal
   14 |     380 |  0.00000E+00 |  0.00000E+00 |      88 |  0.000665530 |            f
   15 |     390 |  0.00000E+00 |  0.00000E+00 |      91 |  0.005416649 |            f
   16 |     400 |  0.00000E+00 |  0.00000E+00 |      87 |  0.006280595 |            f
   17 |     410 |  0.00000E+00 |  0.00000E+00 |      85 |  0.139795051 |        nadir
   18 |     420 |  0.00000E+00 |  0.00000E+00 |      84 |  0.002573418 |            f
   19 |     430 |  0.00000E+00 |  0.00000E+00 |      86 |  0.031658533 |        ideal
   20 |     440 |  0.00000E+00 |  0.00000E+00 |      89 |  0.004441177 |            f
   21 |     450 |  0.00000E+00 |  0.00000E+00 |      8

  106 |    1300 |  0.00000E+00 |  0.00000E+00 |     217 |  0.005426592 |        ideal
  107 |    1310 |  0.00000E+00 |  0.00000E+00 |     214 |  0.001796182 |            f
  108 |    1320 |  0.00000E+00 |  0.00000E+00 |     218 |  0.007839937 |        nadir
  109 |    1330 |  0.00000E+00 |  0.00000E+00 |     212 |  0.010658079 |        nadir
  110 |    1340 |  0.00000E+00 |  0.00000E+00 |     215 |  0.002581493 |            f
  111 |    1350 |  0.00000E+00 |  0.00000E+00 |     219 |  0.001769446 |            f
  112 |    1360 |  0.00000E+00 |  0.00000E+00 |     224 |  0.017135894 |        ideal
  113 |    1370 |  0.00000E+00 |  0.00000E+00 |     229 |  0.031565367 |        ideal
  114 |    1380 |  0.00000E+00 |  0.00000E+00 |     225 |  0.002743140 |            f
  115 |    1390 |  0.00000E+00 |  0.00000E+00 |     229 |  0.001331611 |            f
  116 |    1400 |  0.00000E+00 |  0.00000E+00 |     233 |  0.002330321 |            f
  117 |    1410 |  0.00000E+00 |  0.00000E+00 |     23

In [89]:
print(np.shape(res.X))
print(np.shape(res.pop.get("X")))

(250, 39)
(250, 39)


In [90]:
result=res.pop.get("X")

In [91]:
x_fake_descaled = xscaler.inverse_transform(result)
df = pd.DataFrame(x_fake_descaled, columns=x_orig.columns)
mats=np.full(len(df.index), "Material_Placeholder")
for i in range(len(df.index)):
    if df.loc[i, "Material=Steel"] > df.loc[i, "Material=Aluminum"]:
        if df.loc[i, "Material=Steel"] > df.loc[i, "Material=Titanium"]:
            mats[i]="Steel"
        else:
            mats[i]="Titanium"
    else:
        if df.loc[i, "Material=Aluminum"] > df.loc[i, "Material=Titanium"]:
            mats[i]="Aluminum"
        else:
            mats[i]="Titanium"
df.drop(["Material=Steel", "Material=Aluminum", "Material=Titanium"], axis=1, inplace=True)
df["Material"]=mats

# #The following only if working in augmented space
# import sys
# sys.path.append('../structural')
# import DataAugment
# df=DataAugment.revfeatureAugment(df)

df.to_csv("Generated Data/NSGA2.csv")
print(df)

     SSB_Include  CSB_Include  CS Length   BB Drop     Stack      SS E  \
0       0.937383     0.806807   0.372093  0.024548  0.634018  0.151064   
1       0.824865     0.547449   0.429010  0.024712  0.625202  0.154678   
2       0.572618     0.300399   0.421794 -0.013830  0.628572  0.203988   
3       0.203753     0.791130   0.346039  0.074537  0.630065  0.101179   
4       0.572618     0.290492   0.334934 -0.022269  0.629956  0.174327   
..           ...          ...        ...       ...       ...       ...   
245     0.949863     0.790819   0.346186 -0.004752  0.629383  0.045959   
246     0.920196     0.947879   0.352402 -0.018992  0.628566  0.199604   
247     0.917896     0.672762   0.346062 -0.023229  0.582815  0.198385   
248     0.938963     0.827755   0.339748 -0.018183  0.530923  0.154816   
249     0.829057     0.792232   0.368792 -0.023295  0.626396  0.020124   

      ST Angle     BB OD     TT OD     HT OD  ...      SS Z  SS Thickness  \
0    76.524059  0.046206  0.037090